学生是否获得推荐信
  
Reference : https://blog.csdn.net/qq_49323609/article/details/132856846

# <img src="https://gitee.com/qq_2013977911/picgo-store-images/raw/master/imgs/20250515161227569.png" width="75%">

In [7]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.inference import VariableElimination

from pgmpy.factors.discrete import TabularCPD

In [8]:
letter_bn = DiscreteBayesianNetwork([
       ('D', 'G'),
       ('I', 'G'),
       ('I', 'S'),
       ('G', 'L')
   ])

In [9]:
# 变量名，变量取值个数，对应概率
d_cpd=TabularCPD(variable='D',variable_card=2,values=[[0.6],[0.4]]) # P(D=0)=0.6，P(D=1)=0.4
i_cpd=TabularCPD(variable='I',variable_card=2,values=[[0.7],[0.3]]) # P(I=0)=0.7，P(I=1)=0.3

# 定义变量 G 的条件概率分布
g_cpd=TabularCPD(   
                    variable='G',
                    variable_card=3,
                    # values 每一列对应 I、D 的一种组合（共 2×2=4 种），每一行对应 G 的一个取值
                    values=[
                            [0.3, 0.05, 0.9, 0.5],
                            [0.4, 0.25, 0.08, 0.3],
                            [0.3, 0.7, 0.02, 0.2]
                            ],
                    evidence=['I','D'],  # G 的概率依赖于 I 和 D
                    evidence_card=[2,2]  # I 和 D 都有两个取值
                )
# 变量名，变量取值个数，对应概率，依赖变量名，依赖变量取值，类比 G
s_cpd=TabularCPD(
                    variable='S',
                    variable_card=2,
                    values=[[0.95,0.2],
                            [0.05,0.8]],
                    evidence=['I'],
                    evidence_card=[2]
                )
l_cpd=TabularCPD(
                    variable='L',
                    variable_card=2,
                    values=[[0.1,0.4,0.99],
                            [0.9,0.6,0.01]],
                    evidence=['G'],
                    evidence_card=[3]
                )
# evidence_card必须是列表


In [10]:
# 将前面定义好的所有条件概率分布（CPD）添加到贝叶斯网络 letter_bn 中，
# 只有添加了 CPD，网络才有了“概率信息”，才能进行推理。
letter_bn.add_cpds(d_cpd,i_cpd,g_cpd,s_cpd,l_cpd)

letter_bn.check_model()  # 检查构建的模型是否合理，如果有问题会报错，否则返回 True

letter_bn.get_cpds()  # 获取网络中所有节点的条件概率分布（CPD）对象。

print(letter_bn.check_model()) # 打印check_model()的结果，如果输出 True说明模型结构和概率分布都没问题。

print(letter_bn.get_cpds()) # 打印出网络中所有变量的条件概率分布（CPD），可以直观地看到每个变量的概率表。

True
[<TabularCPD representing P(D:2) at 0x1c13af2ee30>, <TabularCPD representing P(I:2) at 0x1c13af2ef20>, <TabularCPD representing P(G:3 | I:2, D:2) at 0x1c13af2fe20>, <TabularCPD representing P(S:2 | I:2) at 0x1c13af2fd90>, <TabularCPD representing P(L:2 | G:3) at 0x1c13af2ec50>]


True

[<TabularCPD representing P(D:2) at 0x1c13af2ee30>, 
<TabularCPD representing P(I:2) at 0x1c13af2ef20>, 
<TabularCPD representing P(G:3 | I:2, D:2) at 0x1c13af2fe20>, 
<TabularCPD representing P(S:2 | I:2) at 0x1c13af2fd90>, 
<TabularCPD representing P(L:2 | G:3) at 0x1c13af2ec50>]

In [11]:
# 创建一个变量消除（Variable Elimination）推理器，用于在贝叶斯网络 letter_bn 上进行概率推理
letter_infer=VariableElimination(letter_bn) # 变量消除

# 计算在已知 I=1（智力低）且 D=1（课程难）时，L（是否获得推荐信）的概率分布。
prob_I = letter_infer.query( variables=['L'],evidence={'I':1,'D':1} )
print( f"prob_I:{prob_I}" ) # 打印出上述条件下 L 的概率分布结果。

prob_I:+------+----------+
| L    |   phi(L) |
+======+==========+
| L(0) |   0.3680 |
+------+----------+
| L(1) |   0.6320 |
+------+----------+


设置的条件是：
    智力 I=1（低智力）
    课程难度 D=1（难课程）
在这两个条件下，模型推断出：
    该学生获得推荐信的概率为 63.2%
    未获得推荐信的概率为 36.8%